In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import requests
from PIL import Image
import io

# Step 1: Build VGG16 Pretrained Model
def build_vgg16_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers
    x = Flatten()(base_model.output)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_vgg16_model()

# Step 2: Modify the code to use Medical Mask Dataset
# Clone the Medical Mask dataset if not already cloned
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git

def load_dataset():
    data_gen = ImageDataGenerator(
        rescale=1.0/255,
        validation_split=0.2,
    )

    # Set the correct dataset path
    dataset_path = 'Face-Mask-Detection/dataset'  # Adjust based on the repo structure

    train_data = data_gen.flow_from_directory(
        dataset_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )

    val_data = data_gen.flow_from_directory(
        dataset_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )
    return train_data, val_data

train_dataset, val_dataset = load_dataset()


fatal: destination path 'Face-Mask-Detection' already exists and is not an empty directory.
Found 3274 images belonging to 2 classes.
Found 818 images belonging to 2 classes.


In [ ]:
def train_model(model, train_data, val_data, epochs=2):
    model.fit(train_data, validation_data=val_data, epochs=epochs)
    return model

model = train_model(model, train_dataset, val_dataset)

Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 37/103 ━━━━━━━━━━━━━━━━━━━━ 19:29 18s/step - accuracy: 0.7425 - loss: 1.2557

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


103/103 ━━━━━━━━━━━━━━━━━━━━ 2273s 22s/step - accuracy: 0.8495 - loss: 0.6891 - val_accuracy: 0.9584 - val_loss: 0.0843
Epoch 2/2
103/103 ━━━━━━━━━━━━━━━━━━━━ 2257s 22s/step - accuracy: 0.9956 - loss: 0.0185 - val_accuracy: 0.9939 - val_loss: 0.0261


In [ ]:
# Step 3: Classify image from local path
def test_image_from_local(image_path, model, class_names):
    try:
        # Load image from local file
        img = Image.open(image_path).convert('RGB')
        img = img.resize((224, 224))

        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        predictions = model.predict(img_array)
        predicted_class = class_names[np.argmax(predictions)]
        print(f"Predicted class: {predicted_class}")
    except Exception as e:
        print(f"Error: {e}")

# Example usage for local image
test_image_path = r"/content/images/1.jpg"
test_image_from_local(test_image_path, model, ['with_mask', 'without_mask'])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 884ms/step
Predicted class: with_mask
